In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from random import randint

In [2]:
base = pd.read_csv('Base_transformada_maior_que_1.csv')

In [3]:
base.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
base.head()

,Matriz mesangial,Celularidade mesangial,Membrana basal glomerular,Espaço de Bowman,Tufo glomerular,Podócito,Necrose tubular aguda,Vacuolização do epitélio,Cristais,Calcificação,...,Hialinose,Necrose Fibrinóide,IMUNOFLUORESCÊNCIA,padrão de deposição,Imunodepósitos,Imunohistoquímica,Microscopia eletrônica,vermelho congo,Clínica,Diagnóstico
0,1,0,0,1,1,1,0,0,0,0,...,0,0,1,1,1,0,2,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,2,2,1,3,0,0,1
2,2,0,2,1,0,0,0,0,0,0,...,0,0,1,2,2,1,3,0,0,1
3,2,0,3,0,0,0,0,0,0,0,...,0,0,1,2,2,1,3,0,0,1
4,2,0,3,0,0,0,0,0,0,0,...,0,0,1,2,2,1,3,0,0,1


In [5]:
base_by_class = []

for class_i in base['Diagnóstico'].unique():
    base_class_i = base.copy()
    
    target = base_class_i['Diagnóstico']
    base_class_i.drop('Diagnóstico', axis = 1, inplace = True)
    
    target[target != class_i] = -1
    target[target == class_i] = 1
    target[target == -1] = 0
    
    base_class_i['Diagnóstico'] = target
    
    base_by_class.append(base_class_i)

In [6]:
n_hidden1 = 100
n_hidden2 = 50
n_outputs = 2

In [7]:
comite = []

for i in range(len(base_by_class)):
    model = tf.keras.Sequential([tf.keras.layers.Dense(n_hidden1,activation = 'relu'),
                             tf.keras.layers.Dense(n_hidden2,activation = 'relu'),
                             tf.keras.layers.Dense(n_outputs,activation = 'softmax')])
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    comite.append(model)

In [8]:
indexes = []

for i in range(len(base_by_class)):
    base_with_index = base_by_class[i].reset_index()
    
    length = base_with_index[base_with_index['Diagnóstico'] == 1].shape[0]
    
    
    
    index = base_with_index[base_with_index['Diagnóstico'] == 1].iloc[randint(0,length-1)]
    
    indexes.append(index['index'])

In [9]:
len(indexes)/base.shape[0]

0.25287356321839083

In [10]:
base_by_class_train = []
base_by_class_test = []

for i in range(len(base_by_class)):
    train = base_by_class[i].drop(indexes)
    test = base_by_class[i].iloc[indexes]
    
    base_by_class_train.append(train)
    base_by_class_test.append(test)

In [11]:
predictions = []
y_true = []

for i in range(len(base_by_class)):
    x_train = base_by_class_train[i].drop('Diagnóstico', axis = 1)
    y_train = base_by_class_train[i]['Diagnóstico']
    
    comite[i].fit(x = x_train.to_numpy(), y = y_train.to_numpy(), batch_size = 10, epochs = 20)
    
    x_test = base_by_class_test[i].drop('Diagnóstico', axis = 1)
    y_true.append(base_by_class_test[i]['Diagnóstico'].reset_index(drop = True))
    
    y_pred = comite[i].predict(x = x_test.to_numpy(), batch_size = 10)
    predictions.append(y_pred)

Train on 65 samples
Epoch 1/20
65/65 [==============================] - 1s 8ms/sample - loss: 0.6955 - accuracy: 0.6000
Epoch 2/20
65/65 [==============================] - 0s 307us/sample - loss: 0.3007 - accuracy: 0.9385
Epoch 3/20
65/65 [==============================] - 0s 429us/sample - loss: 0.3037 - accuracy: 0.9385
Epoch 4/20
65/65 [==============================] - 0s 261us/sample - loss: 0.2561 - accuracy: 0.9385
Epoch 5/20
65/65 [==============================] - 0s 292us/sample - loss: 0.1858 - accuracy: 0.9385
Epoch 6/20
65/65 [==============================] - 0s 276us/sample - loss: 0.1635 - accuracy: 0.9385
Epoch 7/20
65/65 [==============================] - 0s 230us/sample - loss: 0.1617 - accuracy: 0.9538
Epoch 8/20
65/65 [==============================] - 0s 338us/sample - loss: 0.1389 - accuracy: 0.9385
Epoch 9/20
65/65 [==============================] - 0s 368us/sample - loss: 0.1235 - accuracy: 0.9385
Epoch 10/20
65/65 [==============================] - 0s 215us/sa

65/65 [==============================] - 0s 261us/sample - loss: 0.0390 - accuracy: 0.9846
Train on 65 samples
Epoch 1/20
65/65 [==============================] - 0s 4ms/sample - loss: 0.9281 - accuracy: 0.5077
Epoch 2/20
65/65 [==============================] - 0s 199us/sample - loss: 0.1729 - accuracy: 0.9385
Epoch 3/20
65/65 [==============================] - 0s 171us/sample - loss: 0.1620 - accuracy: 0.9538
Epoch 4/20
65/65 [==============================] - 0s 199us/sample - loss: 0.1831 - accuracy: 0.9538
Epoch 5/20
65/65 [==============================] - 0s 169us/sample - loss: 0.1755 - accuracy: 0.9538
Epoch 6/20
65/65 [==============================] - 0s 154us/sample - loss: 0.1627 - accuracy: 0.9538
Epoch 7/20
65/65 [==============================] - 0s 199us/sample - loss: 0.1517 - accuracy: 0.9538
Epoch 8/20
65/65 [==============================] - 0s 199us/sample - loss: 0.1428 - accuracy: 0.9538
Epoch 9/20
65/65 [==============================] - 0s 184us/sample - loss:

65/65 [==============================] - 0s 169us/sample - loss: 0.0258 - accuracy: 0.9846
Epoch 20/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0230 - accuracy: 0.9846
Train on 65 samples
Epoch 1/20
65/65 [==============================] - 0s 4ms/sample - loss: 0.5359 - accuracy: 0.8615
Epoch 2/20
65/65 [==============================] - 0s 199us/sample - loss: 0.3555 - accuracy: 0.9077
Epoch 3/20
65/65 [==============================] - 0s 199us/sample - loss: 0.2689 - accuracy: 0.9077
Epoch 4/20
65/65 [==============================] - 0s 199us/sample - loss: 0.2420 - accuracy: 0.9077
Epoch 5/20
65/65 [==============================] - 0s 153us/sample - loss: 0.2210 - accuracy: 0.8923
Epoch 6/20
65/65 [==============================] - 0s 185us/sample - loss: 0.1789 - accuracy: 0.9231
Epoch 7/20
65/65 [==============================] - 0s 214us/sample - loss: 0.2083 - accuracy: 0.9077
Epoch 8/20
65/65 [==============================] - 0s 184us/sample - loss

65/65 [==============================] - 0s 199us/sample - loss: 0.0888 - accuracy: 0.9846
Epoch 18/20
65/65 [==============================] - ETA: 0s - loss: 0.0667 - accuracy: 1.00 - 0s 169us/sample - loss: 0.0767 - accuracy: 0.9846
Epoch 19/20
65/65 [==============================] - 0s 169us/sample - loss: 0.0776 - accuracy: 0.9846
Epoch 20/20
65/65 [==============================] - 0s 153us/sample - loss: 0.0768 - accuracy: 0.9846
Train on 65 samples
Epoch 1/20
65/65 [==============================] - 0s 4ms/sample - loss: 0.4503 - accuracy: 0.8769
Epoch 2/20
65/65 [==============================] - 0s 169us/sample - loss: 0.1548 - accuracy: 0.9846
Epoch 3/20
65/65 [==============================] - 0s 169us/sample - loss: 0.1234 - accuracy: 0.9846
Epoch 4/20
65/65 [==============================] - 0s 175us/sample - loss: 0.1006 - accuracy: 0.9846
Epoch 5/20
65/65 [==============================] - 0s 185us/sample - loss: 0.0772 - accuracy: 0.9846
Epoch 6/20
65/65 [============

65/65 [==============================] - 0s 153us/sample - loss: 0.0286 - accuracy: 0.9846
Epoch 16/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0239 - accuracy: 0.9846
Epoch 17/20
65/65 [==============================] - 0s 215us/sample - loss: 0.0202 - accuracy: 0.9846
Epoch 18/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0168 - accuracy: 0.9846
Epoch 19/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0123 - accuracy: 1.0000
Epoch 20/20
65/65 [==============================] - 0s 169us/sample - loss: 0.0113 - accuracy: 1.0000
Train on 65 samples
Epoch 1/20
65/65 [==============================] - 0s 4ms/sample - loss: 0.4600 - accuracy: 0.9538
Epoch 2/20
65/65 [==============================] - 0s 215us/sample - loss: 0.2654 - accuracy: 0.9692
Epoch 3/20
65/65 [==============================] - 0s 199us/sample - loss: 0.1302 - accuracy: 0.9692
Epoch 4/20
65/65 [==============================] - 0s 169us/sample - 

65/65 [==============================] - 0s 169us/sample - loss: 0.0520 - accuracy: 0.9846
Epoch 15/20
65/65 [==============================] - 0s 153us/sample - loss: 0.0964 - accuracy: 0.9538
Epoch 16/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0227 - accuracy: 0.9846
Epoch 17/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0708 - accuracy: 0.9846
Epoch 18/20
65/65 [==============================] - 0s 169us/sample - loss: 0.0630 - accuracy: 0.9846
Epoch 19/20
65/65 [==============================] - 0s 138us/sample - loss: 0.0407 - accuracy: 0.9846
Epoch 20/20
65/65 [==============================] - 0s 184us/sample - loss: 0.0267 - accuracy: 0.9846
Train on 65 samples
Epoch 1/20
65/65 [==============================] - 0s 4ms/sample - loss: 0.4124 - accuracy: 0.8154
Epoch 2/20
65/65 [==============================] - 0s 184us/sample - loss: 0.1446 - accuracy: 0.9846
Epoch 3/20
65/65 [==============================] - 0s 156us/sample -

In [12]:
len(predictions)

22

In [13]:
predictions[0]

array([[9.9656206e-01, 3.4379126e-03],
       [9.7272664e-01, 2.7273279e-02],
       [9.8710793e-01, 1.2892123e-02],
       [9.6618229e-01, 3.3817727e-02],
       [9.7869295e-01, 2.1307025e-02],
       [9.9622101e-01, 3.7789696e-03],
       [9.9452668e-01, 5.4733353e-03],
       [9.9860233e-01, 1.3976903e-03],
       [9.9890721e-01, 1.0927384e-03],
       [9.9160099e-01, 8.3990712e-03],
       [8.4738094e-01, 1.5261900e-01],
       [9.8886442e-01, 1.1135573e-02],
       [9.8899907e-01, 1.1000931e-02],
       [9.9881136e-01, 1.1885937e-03],
       [5.8043176e-01, 4.1956827e-01],
       [9.8743814e-01, 1.2561843e-02],
       [9.9866223e-01, 1.3377392e-03],
       [9.9985671e-01, 1.4324443e-04],
       [9.9925655e-01, 7.4343575e-04],
       [9.4808149e-01, 5.1918533e-02],
       [9.3852377e-01, 6.1476178e-02],
       [9.7821563e-01, 2.1784414e-02]], dtype=float32)

In [14]:
predictions_reshape = np.zeros((22,22), dtype=np.float32)

for i in range(len(predictions)):
    for j in range(predictions[0].shape[0]):
        predictions_reshape[j][i] = predictions[i][j][1]

In [15]:
predictions_argmax = np.zeros(22, dtype=np.uint8)

for i in range(22):
        predictions_argmax[i] = np.argmax(predictions_reshape[i])
#         print(predictions_argmax[i])
#         print(np.argmax(predictions_argmax[i]))

In [16]:
predictions_argmax

array([11,  1,  1,  3,  4,  5,  6, 13,  8,  9, 20, 11, 11, 13,  0, 13, 16,
        1,  3, 11, 20, 13], dtype=uint8)

In [17]:
y_true_reshape = np.zeros((len(y_true)), dtype=np.uint8)

for i in range(len(y_true)):
    for j in range(y_true[0].size):
        if y_true[i][j] == 1:
            y_true_reshape[i] = j
            break

In [18]:
y_true_reshape

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21], dtype=uint8)

In [19]:
m = tf.metrics.Accuracy()
m.update_state(y_true_reshape, predictions_argmax)
print('Final result: ', m.result().numpy())

Final result:  0.5


In [20]:
sum(y_true_reshape == predictions_argmax)/y_true_reshape.size

0.5